[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://github.com/totti0223/phenogarden/blob/main/modules/001_rice_panicle/main.ipynb)

# Preperation

In [ ]:
!pip install -qqq onnxruntime-gpu supervision

!git clone https://github.com/SUNJHZAU/EOPT

In [ ]:
# check if onnxruntime can use gpu
import onnxruntime as ort
ort.get_available_providers()

In [ ]:
import os
import cv2
import copy


from skimage.morphology import skeletonize
import pandas as pd
import numpy as np
import supervision as sv

# Grain Detection

In [ ]:
# code based on https://github.com/SUNJHZAU/EOPT
class YOLOv8:
    """
    YOLOv8による物体検出を行うクラス。
    推論と後処理を担当する。
    """

    def __init__(self, onnx_model: str, confidence_thres: float, iou_thres: float):
        """
        YOLOv8のインスタンスを初期化する。

        Args:
            onnx_model (str): ONNXモデルのパス。
            confidence_thres (float): 検出の信頼度しきい値。
            iou_thres (float): NMS（非最大抑制）のIoUしきい値。
        """
        self.onnx_model = onnx_model
        self.confidence_thres = confidence_thres
        self.iou_thres = iou_thres

        # ONNXモデルを用いて推論セッションを作成。CUDAとCPUの両方を利用する。
        self.session = ort.InferenceSession(self.onnx_model,
                                            providers=["CUDAExecutionProvider", "CPUExecutionProvider"])
        self.model_inputs = self.session.get_inputs()

        # モデルの入力サイズを取得する
        input_shape = self.model_inputs[0].shape
        self.input_width = input_shape[2]
        self.input_height = input_shape[3]

    def preprocess(self, img_path: str) -> np.ndarray:
        """
        画像を推論前に前処理する。

        Args:
            img_path (str): 入力画像のパス。

        Returns:
            np.ndarray: 推論用に前処理された画像データ。
        """
        # OpenCVで画像を読み込み、元画像とそのサイズを保持する
        self.img = cv2.imread(img_path)
        self.img_height, self.img_width = self.img.shape[:2]

        # BGRからRGBに変換し、モデルの入力サイズにリサイズする
        img_rgb = cv2.cvtColor(self.img, cv2.COLOR_BGR2RGB)
        resized_img = cv2.resize(img_rgb, (self.input_width, self.input_height))

        # 画像を0～1の範囲に正規化し、チャンネルファーストに変換する
        image_data = np.array(resized_img) / 255.0
        image_data = np.transpose(image_data, (2, 0, 1))
        image_data = np.expand_dims(image_data, axis=0).astype(np.float32)

        return image_data

    def postprocess(self, output: list) -> list:
        """
        モデルの出力から検出結果（バウンディングボックス、信頼度、クラスID）を抽出する。

        Args:
            output (list): モデルの推論結果。

        Returns:
            list: 検出結果の辞書リスト。各辞書は 'box', 'score', 'clsID' を含む。
        """
        # 出力を整形する（余分な次元を除去し転置する）
        outputs = np.transpose(np.squeeze(output[0]))
        num_rows = outputs.shape[0]

        boxes, scores, class_ids = [], [], []
        x_scale = self.img_width / self.input_width
        y_scale = self.img_height / self.input_height

        # 各出力行について処理する
        for i in range(num_rows):
            row = outputs[i]
            class_scores = row[4:]
            max_score = np.amax(class_scores)

            # 信頼度が閾値以上の検出のみ採用する
            if max_score >= self.confidence_thres:
                cls_id = int(np.argmax(class_scores))
                x_center, y_center, w, h = row[0], row[1], row[2], row[3]
                left = int((x_center - w / 2) * x_scale)
                top = int((y_center - h / 2) * y_scale)
                right = int((x_center + w / 2) * x_scale)
                bottom = int((y_center + h / 2) * y_scale)
                # width = int(w * x_scale)
                # height = int(h * y_scale)
                #boxes.append([left, top, width, height])
                # record in xyxy format instead
                boxes.append([left,top,right,bottom])
                scores.append(float(max_score))
                class_ids.append(cls_id)

        # 非最大抑制を適用して重複検出を排除する
        indices = cv2.dnn.NMSBoxes(boxes, scores, self.confidence_thres, self.iou_thres)
        results = []

        # NMSBoxesが返す形式に注意（場合によっては [[idx], ...] の形式）
        if len(indices) > 0:
            indices = indices.flatten()  # フラットな配列に変換する
            for idx in indices:
                results.append({
                    'xyxy': boxes[idx],
                    'scores': scores[idx],
                    'class_id': class_ids[idx]
                })

        return results

    def __call__(self, img_path: str) -> list:
        """
        画像パスを与えると、前処理・推論・後処理を実行し検出結果を返す。

        Args:
            img_path (str): 入力画像のパス。

        Returns:
            list: 検出結果の辞書リスト。
        """
        # 前処理、推論、後処理の各工程を順に実施する
        img_data = self.preprocess(img_path)
        outputs = self.session.run(None, {self.model_inputs[0].name: img_data})
        return self.postprocess(outputs)


In [ ]:
detector = YOLOv8('GrainNuber.onnx', 0.1, 0.8)

In [ ]:
path = "/content/EOPT/show/cut/1269.jpg"
result = detector(path)

In [ ]:
def from_yolov8(cls, result):
    """
    Creates a Detections instance from yolov8. monkey patch for supervision.
    """
    return cls(
        xyxy=np.array([x["xyxy"] for x in result]),
        confidence=np.array([x["scores"] for x in result]),
        class_id=np.array([x["class_id"] for x in result]),
        tracker_id=None,
        data={"class_name": np.array(["grain"]*len(result))}
    )
sv.Detections.from_yolov8 = classmethod(from_yolov8)

In [ ]:
image = cv2.imread(path)
result = detector(path)

orig_detections = sv.Detections.from_yolov8(result)

In [ ]:
detections = copy.deepcopy(orig_detections[orig_detections.confidence>0.5])

# bounding_box_annotator = sv.BoxAnnotator()
bounding_box_annotator = sv.BoxCornerAnnotator()#sv.RoundBoxAnnotator()
label_annotator = sv.LabelAnnotator(
    text_scale=0.5
)

labels = [
    f"{confidence:.2f}"
    for confidence
    in detections.confidence
]

annotated_image = bounding_box_annotator.annotate(
    scene=image.copy(),
    detections=detections,
)
annotated_image = label_annotator.annotate(
    scene=annotated_image, detections=detections, labels=labels)

annotated_image = sv.draw_text(
    scene=annotated_image, text=str(len(detections.xyxy)),
    text_anchor=sv.Point(x=200,y=100),
    text_color=sv.Color(255,255,255),
    text_scale=5,
    text_thickness=10)
plt.figure(figsize=(20,20))
plt.axis("off")
plt.imshow(annotated_image[...,::-1])